In [1]:
from copy import copy

In [2]:
VAR_INDEX = 0
SUBSTITUDE_INDEX = 0
VARS_DICT = {}

# Abstract class Formula
class Formula(): 
    def __init__(self, args, negated=False):
        self.negated = negated
        self.args = args 
        self.tree_args = []
        self.is_substitude = False
        self.substitude = None
    def __neg__(self): 
        new_object = copy(self)
        new_object.negated = not new_object.negated
        return new_object 
    def __add__(self, other): return Disjunction(self, other)
    def __mul__(self, other): return Conjunction(self, other)
    def __rshift__(self, other): return Implication(self, other)
    def __mod__(self, other): return Equivalence(self, other)
    def __lt__(self, other):        
        if type(self) == Var and type(other) == Var:
            if self.name == other.name:
                return self.negated < other.negated
            return self.name < other.name
        
        self_flattenize = []
        other_flattenize = []
        flattenize(self.tree_args, self_flattenize)
        flattenize(other.tree_args, other_flattenize)
        
        if type(self) == Var and (type(other) == Conjunction or type(other) == Disjunction):
            if is_all_leaves(other):
                return self.name <= other.args[0].name
            return self.name <= other_flattenize[0]
        if  (type(self) == Conjunction or type(self) == Disjunction) and type(other) == Var:
            if is_all_leaves(self):
                return self.args[0].name < other.name
            return self_flattenize[0] < other.name
        if  (type(self) == Conjunction or type(self) == Disjunction) and (type(other) == Conjunction or type(other) == Disjunction):
            if is_all_leaves(self) and is_all_leaves(other):
                return self.args < other.args
            return self_flattenize < other_flattenize
        
        
# Abstract class Connective 
class Connective(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)
        

# Conjunction
class Conjunction(Connective):
    pass

# Disjunction
class Disjunction(Connective): 
    pass

# Implication
class Implication(Formula):
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Equivalence
class Equivalence(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Variable
class Var(Formula):
    def __init__(self, name):
        super().__init__(None)        
        global VAR_INDEX, VARS_DICT
        VAR_INDEX += 1
        VARS_DICT[name] = VAR_INDEX
        self.index = VAR_INDEX
        self.name = name

# Substitute for CNFs        
class Substitute(Formula):
    def __init__(self):
        super().__init__(None)        
        global VAR_INDEX, SUBSTITUDE_INDEX, VARS_DICT
        VAR_INDEX += 1
        SUBSTITUDE_INDEX += 1
        name = 'r' + str(SUBSTITUDE_INDEX)
        VARS_DICT[name] = VAR_INDEX
        self.index = VAR_INDEX
        self.name = name

In [12]:
def remove_imp_eq(formula):
    '''
    removes implications and equivalencies from formula
    returns new formula
    '''
    if type(formula) == Var or type(formula) == Substitute:
        return formula
    expr = None
    if type(formula) == Conjunction:
        expr = remove_imp_eq(formula.args[0])
        for i in range(1, len(formula.args)):
            expr *= remove_imp_eq(formula.args[i])
    
    if type(formula) == Disjunction:
        expr = remove_imp_eq(formula.args[0])
        for i in range(1, len(formula.args)):
            expr += remove_imp_eq(formula.args[i])
    
    if type(formula) == Implication:
        expr = remove_imp_eq(-(formula.args[0])) + remove_imp_eq(formula.args[1])
    
    if type(formula) == Equivalence:
        expr = (remove_imp_eq(-formula.args[0]) + remove_imp_eq(formula.args[1])) * \
               (remove_imp_eq(formula.args[0]) + remove_imp_eq(-formula.args[1]))
    expr.negated = formula.negated
    return expr

def de_morgan(formula):
    '''
    applies de Morgan rule to formula
    return new formula
    '''
    if type(formula) == Var or type(formula) == Substitute:
        return formula
    if type(formula) == Disjunction :
        if formula.negated:
            new_formula = de_morgan(-formula.args[0])
            for i in range(1, len(formula.args)):
                new_formula *= de_morgan(-formula.args[i])
            return new_formula
        else:
            new_formula = de_morgan(formula.args[0])
            for i in range(1, len(formula.args)):
                new_formula += de_morgan(formula.args[i])
            return new_formula
            
    if type(formula) == Conjunction:
        if formula.negated:
            new_formula = de_morgan(-formula.args[0])
            for i in range(1, len(formula.args)):
                new_formula += de_morgan(-formula.args[i])
            return new_formula
        else:
            new_formula = de_morgan(formula.args[0])
            for i in range(1, len(formula.args)):
                new_formula *= de_morgan(formula.args[i])
            return new_formula

def flatten_list(l):
    '''
    max depth of l is 2
    returns flatten list l
    '''
    flat_list = []
    for list_elem in l:
        if type(list_elem) == list:
            for item in list_elem:
                flat_list.append(item)
        else:
            flat_list.append(list_elem)
    return flat_list

def remove_associativity(formula):
    '''
    removes disjunction and conjunction associativity
    changes formula
    '''
    if type(formula) != Var and type(formula) != Substitute: 
        for i, arg in enumerate(formula.args):
            if (type(formula) == Disjunction and type(formula.args[i]) == Disjunction) or \
            (type(formula) == Conjunction and type(formula.args[i]) == Conjunction):
                formula.args[i] = formula.args[i].args
                formula.args = flatten_list(formula.args)
                remove_associativity(formula)  
        for i, arg in enumerate(formula.args):
            remove_associativity(arg)

def get_tree_args(formula):
    '''
    get arguments of formula without root in tree structure
    changes formula
    '''
    if type(formula) != Var and type(formula) != Substitute:
        for arg in formula.args:
            get_tree_args(arg)
            if type(arg) == Conjunction or type(arg) == Disjunction:
                for inner_arg in arg.args:
                    if type(inner_arg) == Var:
                        if inner_arg.negated:
                            inner_arg.tree_args = '-' + inner_arg.name
                            arg.tree_args += ['-' + inner_arg.name]
                        else:
                            inner_arg.tree_args = inner_arg.name
                            arg.tree_args += [inner_arg.name]
                    else:
                        arg.tree_args += [inner_arg.tree_args]
    else:
        if formula.negated:
            formula.tree_args = '-' + formula.name
        else:
            formula.tree_args = formula.name
                        
def get_root_tree(formula):
    '''
    fills arguments of root
    '''
    for arg in formula.args:
        formula.tree_args.append(arg.tree_args)
        
def get_formula_tree(formula):
    '''
    get arguments of formula in tree structure
    changes formula
    '''
    get_tree_args(formula)
    get_root_tree(formula)

    
def clear_formula_tree(formula):
    '''
    clears formula's tree arguments
    '''
    formula.tree_args = []
    if type(formula) != Var and type(formula) != Substitute:
        for arg in formula.args:
            clear_formula_tree(arg)

def is_all_leaves(formula):
    '''
    returns True if all arguments of formula are Vars or Subs
    '''
    for arg in formula.args:
        if type(arg) != Var and type(arg) != Substitute and not arg.is_substitude:
            return False
    return True

def flattenize(tree_args, result):
    '''
    flattens list of various depth, result stores in result
    '''
    for i, elem in enumerate(tree_args):
        if type(elem) != list:
            result += [elem]
        else:
            flattenize(elem, result)

def sorting(formula):
    '''
    formula sorting
    changes formula
    '''
    if type(formula) != Var:
        for arg in formula.args:
            sorting(arg)
        if type(formula) == Conjunction or type(formula) == Disjunction:
            formula.args = sorted(formula.args)
            clear_formula_tree(formula)
            get_formula_tree(formula)                

def find_substitutes(formula):
    '''
    finds nodes of formula to replace with substitutes
    changes formula
    '''
    if type(formula) != Var:
        for arg in formula.args:
            find_substitutes(arg)
        if (type(formula) == Conjunction or type(formula) == Disjunction) and is_all_leaves(formula):
            formula.is_substitude = True
 
            
def replace_substitudes(formula, R):
    '''
    replaces founded nodes of formula and creates R set
    result stores in R
    changes formula
    '''
    if type(formula) != Var:
        for i, arg in enumerate(formula.args):
            replace_substitudes(arg, R)
        for i, arg in enumerate(formula.args):
            if arg.is_substitude:
                sub = Substitute()
                R.append(sub % arg)
                formula.args[i] = sub

def replace_root_substitude(formula, R):
    '''
    replaces root of formula and creates R set
    result stores in R
    changes formula
    '''
    if formula.is_substitude:
        sub = Substitute()
        R.append(sub % formula)
        return sub 
    
def connect_disjuncts(formula):
    '''
    creates conjuction of equivalencies with substitudes variables
    changes formula
    returns new formula
    '''
    find_substitutes(formula)
    R = []
    replace_substitudes(formula, R)
    f = replace_root_substitude(formula, R)
    for r in R:
        f *= r
    return f

def distribute(formula):
    '''
    applies distribution to conjunctions and disjuntions in formula
    changes formula
    '''
    for i, disj in enumerate(formula.args):  
        if type(disj) != Substitute:
            if type(disj.args[1]) == Conjunction:
                new_arg = []
                for conj in disj.args[1].args:
                    new_arg.append(conj)
                new_disj = new_arg[0] + disj.args[0]
                if len(new_arg) > 1:
                    for j in range(1, len(new_arg)):
                        new_disj *= new_arg[j]  + disj.args[0]
                formula.args[i] = new_disj
                
def CNF(f):
    '''
    returns CNF form of f
    changes f
    '''
    f = de_morgan(f)
    remove_associativity(f)
    get_formula_tree(f)
    sorting(f)
    R = []
    f = connect_disjuncts(f)
    f = remove_imp_eq(f)
    f = de_morgan(f)
    remove_associativity(f)
    distribute(f)
    remove_associativity(f)
    clear_formula_tree(f)
    get_formula_tree(f)
    return f

def DIMACS(cnf):
    '''
    returns DIMACS format of CNF
    '''
    dimacs = ''
    disjuncts = cnf.tree_args
    M = len(disjuncts)
    unique_vars = set()
    for var in flatten_list(disjuncts):
        if var[0] == '-':
            unique_vars.add(var[1:])
        else:
            unique_vars.add(var)
    unique_vars = sorted(unique_vars)
    N = len(unique_vars)
    for var in unique_vars:
        dimacs += 'c var     name: ' + var + '  ' + 'index: ' + str(VARS_DICT[var]) + '\n'
    dimacs += 'c\nc\np cnf ' + str(N) + ' ' + str(M) + '\n'
    for disj in disjuncts:
        line = ''
        if type(disj) == list:
            for var in disj:
                if var[0] == '-':
                    line += '-' + str(VARS_DICT[var[1:]]) + ' '
                else:
                    line += str(VARS_DICT[var]) + ' '
        else:
            if disj[0] == '-':
                line += '-' + str(VARS_DICT[disj[1:]]) + ' '
            else:
                line += str(VARS_DICT[disj]) + ' '
        dimacs += line + '0\n'
    return dimacs

In [13]:
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')
e = Var('e')
g = Var('g')
f = (d+c+e*(d+a)*((g*g) + a)) * (a*g + c*c) * e * -(b + a + c)
f = CNF(f)
dimacs_format = DIMACS(f)
print(dimacs_format)
with open('Output.txt', 'w') as text_file:
    text_file.write(dimacs_format)

c var     name: a  index: 53
c var     name: b  index: 54
c var     name: c  index: 55
c var     name: d  index: 56
c var     name: e  index: 57
c var     name: g  index: 58
c var     name: r29  index: 59
c var     name: r30  index: 60
c var     name: r31  index: 61
c var     name: r32  index: 62
c var     name: r33  index: 63
c var     name: r34  index: 64
c var     name: r35  index: 65
c var     name: r36  index: 66
c var     name: r37  index: 67
c
c
p cnf 15 34
67 0
58 -59 0
58 -59 0
59 -58 -58 0
-60 53 56 0
-53 60 0
-56 60 0
-61 53 59 0
-53 61 0
-59 61 0
60 -62 0
61 -62 0
57 -62 0
62 -60 -61 -57 0
53 -63 0
58 -63 0
63 -53 -58 0
55 -64 0
55 -64 0
64 -55 -55 0
-65 62 55 56 0
-62 65 0
-55 65 0
-56 65 0
-66 63 64 0
-63 66 0
-64 66 0
-53 -67 0
65 -67 0
66 -67 0
-54 -67 0
-55 -67 0
57 -67 0
67 53 -65 -66 54 55 -57 0

